<a href="https://colab.research.google.com/github/SachinScaler/UnsupervisedML_Feb24/blob/main/KMeans%2B%2B_Case_Study_Stock_Portfolio_with_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Problem Statement: Building Stock Portfolios
**Goal** : Creating pools of stocks that move together(similar characteristics) on one or more fundamental financial metric to build stock portfolios.

Let's first have a look at the dataset that we'll be using.

Dataset - Top 50 NSE stocks
Contains company's stock symbol
Name, Industry and other details
Lets import the dependencies



In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
plt.rcParams["figure.figsize"] = (18,10)
import seaborn as sns

In [2]:
# download the dataset
!wget "https://drive.google.com/uc?export=download&id=1giO5bbp3l0INVvTQIGJ7s_Ai5_TWNuIb" -O ind_nifty50list.csv

--2024-02-22 11:06:35--  https://drive.google.com/uc?export=download&id=1giO5bbp3l0INVvTQIGJ7s_Ai5_TWNuIb
Resolving drive.google.com (drive.google.com)... 142.251.2.101, 142.251.2.139, 142.251.2.138, ...
Connecting to drive.google.com (drive.google.com)|142.251.2.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1giO5bbp3l0INVvTQIGJ7s_Ai5_TWNuIb&export=download [following]
--2024-02-22 11:06:35--  https://drive.usercontent.google.com/download?id=1giO5bbp3l0INVvTQIGJ7s_Ai5_TWNuIb&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.250.141.132, 2607:f8b0:4023:c0b::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.250.141.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3053 (3.0K) [application/octet-stream]
Saving to: ‘ind_nifty50list.csv’

ind_nifty50list.csv 100%[===================>]   2.98K  --.-KB/s

#### Installing yfinance

yfinance is a library that helps us download market data from yahoo finance's api.

So lets install these libraries into our environment using pip.

You can read more about it here: https://pypi.org/project/yfinance/

In [3]:
!pip install yfinance

# Since yfinance is an old library, it also has some glitches, so we install this library as a fix
!pip install fix-yahoo-finance

In [4]:
stocks_df = pd.read_csv("./ind_nifty50list.csv")
list_of_symbols = list(stocks_df['Symbol'])
stocks_df.head()

,Company Name,Industry,Symbol,Series,ISIN Code
0,Adani Ports and Special Economic Zone Ltd.,SERVICES,ADANIPORTS,EQ,INE742F01042
1,Asian Paints Ltd.,CONSUMER GOODS,ASIANPAINT,EQ,INE021A01026
2,Axis Bank Ltd.,FINANCIAL SERVICES,AXISBANK,EQ,INE238A01034
3,Bajaj Auto Ltd.,AUTOMOBILE,BAJAJ-AUTO,EQ,INE917I01010
4,Bajaj Finance Ltd.,FINANCIAL SERVICES,BAJFINANCE,EQ,INE296A01024


Now, we'll take symbols from original dataset and add .NS at the end of every symbol to fetch data from Yahoo Finance based on company's symbol as they store data in Symbol.NS key


In [5]:
yf_symbols = list(map(lambda x: x + '.NS', list_of_symbols))
yf_symbols

['ADANIPORTS.NS',
 'ASIANPAINT.NS',
 'AXISBANK.NS',
 'BAJAJ-AUTO.NS',
 'BAJFINANCE.NS',
 'BAJAJFINSV.NS',
 'BPCL.NS',
 'BHARTIARTL.NS',
 'BRITANNIA.NS',
 'CIPLA.NS',
 'COALINDIA.NS',
 'DIVISLAB.NS',
 'DRREDDY.NS',
 'EICHERMOT.NS',
 'GRASIM.NS',
 'HCLTECH.NS',
 'HDFCBANK.NS',
 'HDFCLIFE.NS',
 'HEROMOTOCO.NS',
 'HINDALCO.NS',
 'HINDUNILVR.NS',
 'HDFC.NS',
 'ICICIBANK.NS',
 'ITC.NS',
 'IOC.NS',
 'INDUSINDBK.NS',
 'INFY.NS',
 'JSWSTEEL.NS',
 'KOTAKBANK.NS',
 'LT.NS',
 'M&M.NS',
 'MARUTI.NS',
 'NTPC.NS',
 'NESTLEIND.NS',
 'ONGC.NS',
 'POWERGRID.NS',
 'RELIANCE.NS',
 'SBILIFE.NS',
 'SHREECEM.NS',
 'SBIN.NS',
 'SUNPHARMA.NS',
 'TCS.NS',
 'TATACONSUM.NS',
 'TATAMOTORS.NS',
 'TATASTEEL.NS',
 'TECHM.NS',
 'TITAN.NS',
 'UPL.NS',
 'ULTRACEMCO.NS',
 'WIPRO.NS']

Now, we'll define some features/indicators as variables and fetch the corresponding data from Yahoo Finance for the companies in our list

Note:
- Below given given takes a lot of time, Run in Advance.



In [6]:
import yfinance as yf


stock_financials = {
    'marketCap': [],
    'regularMarketVolume': [],
    'earningsQuarterlyGrowth': [],
    'bookValue': [],
    'totalRevenue': [],
    'returnOnAssets': [],
    'profitMargins': [],
    'earningsGrowth': []
    }

for ticker in yf_symbols:
    stock_info = yf.Ticker(ticker).info
    stock_financials['marketCap'].append(stock_info.get('marketCap', None))
    stock_financials['regularMarketVolume'].append(stock_info.get('regularMarketVolume', None))
    stock_financials['earningsQuarterlyGrowth'].append(stock_info.get('earningsQuarterlyGrowth', None))
    stock_financials['bookValue'].append(stock_info.get('bookValue', None))
    stock_financials['totalRevenue'].append(stock_info.get('totalRevenue', None))
    stock_financials['returnOnAssets'].append(stock_info.get('returnOnAssets', None))
    stock_financials['profitMargins'].append(stock_info.get('profitMargins', None))
    stock_financials['earningsGrowth'].append(stock_info.get('earningsGrowth', None))

/usr/local/lib/python3.10/dist-packages/yfinance/base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


**Q. What have we collected above?**
We 've collected attributes such as 'marketCap', 'regularMarketVolume' and others for companies that we collected in variable 'yf_symbols'

Let's verify


In [7]:
df = pd.DataFrame(stock_financials)
df.head()

,marketCap,regularMarketVolume,earningsQuarterlyGrowth,bookValue,totalRevenue,returnOnAssets,profitMargins,earningsGrowth
0,2832936468480,2555332,0.679,178.814,256109101056,NaN,0.28230,0.678
1,2893209665536,2615297,0.350,172.706,354908110848,NaN,0.15321,0.349
2,3396705976320,14632445,0.049,484.056,622467022848,0.01008,0.21568,0.053
3,2406591234048,548170,0.380,1036.725,432663396352,NaN,0.17107,0.380
4,4129618132992,1022158,0.224,989.308,301745700864,NaN,0.45682,0.209


In [8]:
df.shape


(50, 8)

In [9]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   marketCap                50 non-null     int64  
 1   regularMarketVolume      50 non-null     int64  
 2   earningsQuarterlyGrowth  48 non-null     float64
 3   bookValue                49 non-null     float64
 4   totalRevenue             50 non-null     int64  
 5   returnOnAssets           11 non-null     float64
 6   profitMargins            50 non-null     float64
 7   earningsGrowth           47 non-null     float64
dtypes: float64(5), int64(3)
memory usage: 3.2 KB


### Downloading stock price data

- Purpose of this data: Getting returns of the stocks

In [10]:
import yfinance as yf

stock_prices = yf.download(yf_symbols, start='2020-01-01')['Adj Close']
stock_prices.columns = list_of_symbols

[*********************100%%**********************]  50 of 50 completed


- 'Adj Close' refers to Adjusted close which is used in stock market. It is the closing price after adjustments for all applicable splits and dividend distributions

In [11]:
stock_prices.shape

(1029, 50)

In [12]:
# last 5 rows of stock prices
stock_prices.tail()

,ADANIPORTS,ASIANPAINT,AXISBANK,BAJAJ-AUTO,BAJFINANCE,BAJAJFINSV,BPCL,BHARTIARTL,BRITANNIA,CIPLA,...,SUNPHARMA,TCS,TATACONSUM,TATAMOTORS,TATASTEEL,TECHM,TITAN,UPL,ULTRACEMCO,WIPRO
Date,,,,,,,,,,,,,,,,,,,,,
2024-02-16,1306.849976,3007.899902,1068.750000,8325.049805,1575.900024,6619.200195,1120.000000,654.599976,4913.700195,1440.699951,...,1509.949951,1147.449951,938.599976,142.300003,4128.299805,1302.550049,3655.399902,9916.200195,490.899994,543.000000
2024-02-19,1307.000000,2998.100098,1062.949951,8509.700195,1620.150024,6712.799805,1142.199951,652.549988,4925.250000,1470.349976,...,1530.099976,1155.000000,932.599976,141.949997,4103.799805,1310.000000,3695.050049,9902.200195,497.549988,535.950012
2024-02-20,1300.500000,3011.649902,1088.599976,8295.200195,1602.849976,6763.600098,1143.949951,657.599976,4921.149902,1451.650024,...,1538.699951,1154.400024,926.349976,141.050003,4030.649902,1320.550049,3686.699951,9978.950195,498.049988,531.950012
2024-02-21,1291.000000,3001.800049,1084.199951,8232.250000,1586.949951,6688.799805,1139.900024,633.200012,4923.000000,1447.300049,...,1543.400024,1160.750000,921.049988,143.899994,3991.500000,1296.599976,3656.350098,9968.400391,492.149994,521.400024
2024-02-22,1310.949951,3017.399902,1100.900024,8498.450195,1592.550049,6674.950195,1135.550049,625.900024,4956.950195,1453.300049,...,1558.050049,1159.500000,932.299988,145.899994,4087.100098,1328.099976,3652.050049,9961.200195,492.299988,531.049988


In [13]:
# splitting data by year - 2020 returns
price_2020 = stock_prices.loc["2020-01-02 00:00:00":"2020-12-31 00:00:00"]

# to see % growth of the stock: ((endDate/startDate) - 1) * 100
stock_prices.loc['returns_2020'] = (price_2020.loc['2020-08-04 00:00:00'] / price_2020.loc['2020-01-02 00:00:00'] - 1)*100

stock_prices.head()

,ADANIPORTS,ASIANPAINT,AXISBANK,BAJAJ-AUTO,BAJFINANCE,BAJAJFINSV,BPCL,BHARTIARTL,BRITANNIA,CIPLA,...,SUNPHARMA,TCS,TATACONSUM,TATAMOTORS,TATASTEEL,TECHM,TITAN,UPL,ULTRACEMCO,WIPRO
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-01 00:00:00,365.911530,1743.498413,746.799988,2644.205322,935.767700,4164.032715,439.373932,372.358154,2787.020752,466.001068,...,413.836639,313.626343,183.876907,40.974758,1999.319214,642.186218,1139.830933,3991.287109,557.155457,242.605286
2020-01-02 00:00:00,371.240601,1741.019165,755.029114,2619.778564,948.454590,4178.548340,441.215576,369.252930,2799.627930,463.651001,...,414.455994,313.142151,193.148010,42.472713,1990.140869,645.514832,1140.670044,4167.827148,563.974792,243.192963
2020-01-03 00:00:00,370.610779,1702.857300,741.064636,2578.689941,932.515015,4126.784668,441.118622,366.185608,2786.241455,460.174866,...,423.651337,308.154724,190.506256,42.371971,2029.802734,653.140747,1124.975464,4142.690918,559.523254,245.935394
2020-01-06 00:00:00,368.382294,1659.833618,721.414612,2549.814209,902.267151,3933.162354,435.836060,355.317444,2770.883545,457.041443,...,419.220428,302.247253,185.073181,41.456554,2029.618164,649.180298,1143.631348,4081.717285,554.029846,246.963776
2020-01-07 00:00:00,372.839294,1676.605347,723.908264,2550.065918,907.550354,3943.938232,431.425842,349.144989,2784.224121,458.852936,...,425.366486,304.958862,184.126129,41.706207,2034.599121,654.826111,1144.963989,4165.175781,563.595947,249.951050


In [14]:
stock_prices = stock_prices.transpose()
stock_prices.head()

Date,2020-01-01 00:00:00,2020-01-02 00:00:00,2020-01-03 00:00:00,2020-01-06 00:00:00,2020-01-07 00:00:00,2020-01-08 00:00:00,2020-01-09 00:00:00,2020-01-10 00:00:00,2020-01-13 00:00:00,2020-01-14 00:00:00,...,2024-02-12 00:00:00,2024-02-13 00:00:00,2024-02-14 00:00:00,2024-02-15 00:00:00,2024-02-16 00:00:00,2024-02-19 00:00:00,2024-02-20 00:00:00,2024-02-21 00:00:00,2024-02-22 00:00:00,returns_2020
ADANIPORTS,365.911530,371.240601,370.610779,368.382294,372.839294,373.662903,379.621704,379.863953,378.216797,377.102539,...,1249.349976,1264.800049,1267.849976,1267.050049,1306.849976,1307.000000,1300.500000,1291.000000,1310.949951,-16.891375
ASIANPAINT,1743.498413,1741.019165,1702.857300,1659.833618,1676.605347,1680.883545,1723.420776,1742.866699,1754.874146,1768.729370,...,2953.800049,2971.600098,2976.399902,3016.550049,3007.899902,2998.100098,3011.649902,3001.800049,3017.399902,-3.813719
AXISBANK,746.799988,755.029114,741.064636,721.414612,723.908264,722.661438,740.964844,738.171936,735.528748,746.002014,...,1046.699951,1070.750000,1096.849976,1072.199951,1068.750000,1062.949951,1088.599976,1084.199951,1100.900024,-43.305377
BAJAJ-AUTO,2644.205322,2619.778564,2578.689941,2549.814209,2550.065918,2567.945557,2590.147705,2603.158447,2597.156738,2603.494629,...,7829.950195,7917.399902,8083.299805,8123.049805,8325.049805,8509.700195,8295.200195,8232.250000,8498.450195,-0.566445
BAJFINANCE,935.767700,948.454590,932.515015,902.267151,907.550354,912.524353,937.444092,935.149170,943.430786,953.368652,...,1557.250000,1568.000000,1579.699951,1569.400024,1575.900024,1620.150024,1602.849976,1586.949951,1592.550049,-35.138673


In [15]:
# putting it all together in dataframe
prices = stock_prices.iloc[:, -1]
df.index = stock_prices.index
df['return_2020'] = prices
df.head()

,marketCap,regularMarketVolume,earningsQuarterlyGrowth,bookValue,totalRevenue,returnOnAssets,profitMargins,earningsGrowth,return_2020
ADANIPORTS,2832936468480,2555332,0.679,178.814,256109101056,NaN,0.28230,0.678,-16.891375
ASIANPAINT,2893209665536,2615297,0.350,172.706,354908110848,NaN,0.15321,0.349,-3.813719
AXISBANK,3396705976320,14632445,0.049,484.056,622467022848,0.01008,0.21568,0.053,-43.305377
BAJAJ-AUTO,2406591234048,548170,0.380,1036.725,432663396352,NaN,0.17107,0.380,-0.566445
BAJFINANCE,4129618132992,1022158,0.224,989.308,301745700864,NaN,0.45682,0.209,-35.138673


In [16]:
# check for null values
df.isna().sum()

marketCap                   0
regularMarketVolume         0
earningsQuarterlyGrowth     2
bookValue                   1
totalRevenue                0
returnOnAssets             39
profitMargins               0
earningsGrowth              3
return_2020                 1
dtype: int64

In [17]:
df['returnOnAssets'] = df['returnOnAssets'].replace(np.nan, 0)

In [18]:
df.dropna(axis=0, inplace=True)
df.shape

(45, 9)

**Q. Should we Scale the values?**

- The data seems to be in different scales
Different financial metrics have different scales.
- We need to put them on the same scale.
- We can use StandardScaler from sklearn

In [19]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(df)
X = scaler.transform(df)

In [20]:
scaled_df = pd.DataFrame(X, columns=df.columns, index=df.index)

In [21]:
scaled_df

,marketCap,regularMarketVolume,earningsQuarterlyGrowth,bookValue,totalRevenue,returnOnAssets,profitMargins,earningsGrowth,return_2020
ADANIPORTS,-0.221758,-0.505691,0.109108,-0.467325,-0.583201,-0.403534,1.330562,0.109598,-0.403618
ASIANPAINT,-0.204570,-0.499586,-0.088222,-0.474248,-0.530702,-0.403534,0.005802,-0.087511,0.051361
AXISBANK,-0.060992,0.723856,-0.268758,-0.121322,-0.388528,-0.238697,0.646888,-0.264850,-1.322575
BAJAJ-AUTO,-0.343335,-0.710036,-0.070229,0.505148,-0.489385,-0.403534,0.189086,-0.068938,0.164335
BAJFINANCE,0.148006,-0.661780,-0.163795,0.451399,-0.558951,-0.403534,3.121540,-0.171387,-1.038451
BAJAJFINSV,-0.306576,-0.648828,-0.171593,-0.308288,-0.185144,-0.403534,-0.770336,-0.179176,-0.637789
BPCL,-0.643557,0.641692,0.194278,-0.292188,1.670000,-0.403534,-0.906004,0.195871,0.900947
BHARTIARTL,0.880970,0.114015,0.024538,-0.513277,0.069228,0.498162,-0.985537,0.002956,-0.220060
BRITANNIA,-0.689117,-0.739852,-0.539862,-0.535905,-0.630742,-0.403534,-0.236184,-0.538047,1.102968
CIPLA,-0.694966,-0.604729,-0.107415,-0.321452,-0.585829,-0.403534,-0.051359,-0.106084,1.963643
